In [ ]:
%load_ext autoreload
%autoreload 2

import os
import tarfile
import urllib
from pathlib import Path

import matplotlib.pyplot as plt
import polars as pl
import pyarrow
from constants import PROJECT_ROOT

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = PROJECT_ROOT / os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

In [ ]:
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()


fetch_housing_data()

In [ ]:
def load_data(filename: str, housing_path: Path = HOUSING_PATH) -> pl.DataFrame:
    return pl.read_csv(housing_path / filename)


df = load_data("housing.csv")

In [ ]:
df.head(5)

In [ ]:
print(df.shape)
print(df.schema)
df.null_count()

In [ ]:
print(df.get_column("ocean_proximity").value_counts().sort(by=pl.col("count"), descending=True))

In [ ]:
df.describe()

In [ ]:
df.to_pandas().hist(bins=50, figsize=(12, 10))
plt.show()